# Fetch stats from piwik

- uses the public piwik API
- fetch top visited datasets on a given period
- notifies through Slack
- entrypoint: compute(period, date, title='')

In [ ]:
import requests

In [ ]:
import os
from pathlib import Path
current_path = Path(os.getenv('WORKING_DIR')) if os.getenv('WORKING_DIR') else Path().absolute()

In [ ]:
# load config
import configparser
config = configparser.ConfigParser()
config.read(current_path / 'config.ini')
assert 'secrets' in config.sections()

wh_url = config.get('secrets', 'slack_webhook_url')

In [ ]:
base_url = 'https://stats.data.gouv.fr/index.php'

In [ ]:
params = {
    'module': 'API',
    'idSite': 109,
    'date': 'yesterday',
    'period': 'day',
    'format': 'json',
    'method': 'Actions.getPageUrls',
    'expanded': 1,
    'flat': 1,
    'filter_limit': 10,
    'filter_column': 'label',
    'filter_pattern': '/fr/datasets/'
}

In [ ]:
def compute(period, date, title=''):
    params['period'] = period
    params['date'] = date
    r = requests.get(base_url, params=params)
    databits = []
    for data in r.json():
        if 'url' in data:
            databits.append(data)
            print(str(data['nb_visits']).ljust(10), data['url'])
    send_message(databits, title)

In [ ]:
def send_message(databits, title):
    message = []
    for data in databits:
        url = f"<{data['url']}|{data['url'].split('/')[-2]}>"
        message.append(f"`{data['nb_visits']}`".ljust(8) + url)
    message = '\n'.join(message)
    message = f"-- *{title}* --\n{message}"
    requests.post(wh_url, json={'text': message})